In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://books.toscrape.com/catalogue/page-{}.html"

titles, prices, availability, ratings = [], [], [], []

page = 1
while True:
    url = base_url.format(page)
    response = requests.get(url)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")
    if not books:
        break

    for book in books:
       
        title = book.h3.a["title"]
        titles.append(title)
        price = book.find("p", class_="price_color").text.strip()
        prices.append(price)
        avail = book.find("p", class_="instock availability").text.strip()
        availability.append(avail)
        star = book.find("p")["class"][1]
        ratings.append(star)

        
       

        
      

    page += 1


df_books = pd.DataFrame({
    "Title": titles,
    "Price": prices,
    "Availability": availability,
    "Star Rating": ratings
})

df_books.to_csv("books.csv", index=False)
print("Scraping completed. Saved as books.csv")


KeyboardInterrupt: 

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://www.imdb.com/chart/top/")


movie_container = driver.find_elements(By.CSS_SELECTOR, "div.sc-15ac7568-0.jQHOho.cli-children")

movie_list = []


for container in movie_container:
    rank,name = container.find_element(By.CSS_SELECTOR, "h3").text.split(".",1) #split into two parts by first .(dot char)
    release_year = container.find_element(By.CSS_SELECTOR,"span.sc-15ac7568-7.cCsint.cli-title-metadata-item").text
    rating = container.find_element(By.CSS_SELECTOR, "span.ipc-rating-star--rating").text
    
    d = dict()
    d = {
        "Rank": rank.strip(),
        "Title":name.strip(),
        "Year of Releas": release_year,
        "IMDB Rating": rating
    }
    movie_list.append(d)
    
driver.quit()
pd.DataFrame(movie_list).to_csv(path_or_buf="imdb_top250.csv", index=False)
print("Movie List saved successfully, imdb_top250.csv")

Movie List saved successfully, imdb_top250.csv


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome()
driver.get("https://www.timeanddate.com/weather/")

rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")

city_temp_list = []



def extract_city_data(tds, col_start):
    try:
        city_name = tds[col_start].text.strip()
        if not city_name:  
            return None

        condition = tds[col_start+2].find_element(By.CSS_SELECTOR, "img").get_attribute("title") \
                    if tds[col_start+2].find_elements(By.CSS_SELECTOR, "img") else "N/A"

        temperature = tds[col_start+3].text.strip() if tds[col_start+3].text.strip() else "N/A"

        return {
            "City": city_name,
            "Temperature": temperature,
            "Weather Condition": condition
        }
    except Exception:
        return None

for j in range(0,12,4):
    for row in rows:
        tds = row.find_elements(By.CSS_SELECTOR, "td")
        if len(tds) >= 4:
            d = extract_city_data(tds, j)
            if d:
                print(f"Saving {d}")
                city_temp_list.append(d)
driver.quit()


pd.DataFrame(city_temp_list).to_csv("weather.csv", index=False)
print(f"{len(city_temp_list)} cities' temperatures saved successfully, as weather.csv")

Saving {'City': 'Accra', 'Temperature': '24 °C', 'Weather Condition': 'Overcast. Mild.'}
Saving {'City': 'Addis Ababa', 'Temperature': '15 °C', 'Weather Condition': 'Fog. Cool.'}
Saving {'City': 'Adelaide', 'Temperature': '20 °C', 'Weather Condition': 'Refreshingly cool.'}
Saving {'City': 'Algiers', 'Temperature': '23 °C', 'Weather Condition': 'Scattered clouds. Mild.'}
Saving {'City': 'Almaty', 'Temperature': '27 °C', 'Weather Condition': 'Clear. Pleasantly warm.'}
Saving {'City': 'Amman', 'Temperature': '24 °C', 'Weather Condition': 'Haze. Mild.'}
Saving {'City': 'Amsterdam *', 'Temperature': '16 °C', 'Weather Condition': 'Passing clouds. Cool.'}
Saving {'City': 'Anadyr', 'Temperature': '10 °C', 'Weather Condition': 'Overcast. Cool.'}
Saving {'City': 'Anchorage *', 'Temperature': '13 °C', 'Weather Condition': 'Overcast. Cool.'}
Saving {'City': 'Ankara', 'Temperature': '23 °C', 'Weather Condition': 'Sunny. Mild.'}
Saving {'City': 'Antananarivo', 'Temperature': '17 °C', 'Weather Condit